In synchronous programming, when a task starts another task the current task is blocked, it must wait until the new task completes before it can continue. This reduces throughput and user satisfaction. With asynchrovous programming when we start another task, the current task can continue, the new task can run in the background. With this the current task can continue other work, provided it does not need the data from the asynchronous task, this increases the throughtput. Blocking is undesirable even in multi-threaded programs, asynchronous programming reduces the need to block. Synchronization in synchronous programming is achieved with mutexes and atomic operations. Synchronization in asynchronous programming is achieved with message queues. Threads which want thier tasks to be processed, push the tasks into the message queue and continue, another thread monitors the queue and processes the tasks. Asynchronous programming is used for parallel programming, start new threads which all perform the same task, collect the result from each thread as it completes its task and combine the results into the final answer.

## Packaged Task
std::packaged_task class encapsulates every thing that is needed to execute a task, a callable object for the task's code and std::promise for the result of the task. It is a templated class with the callable object's signature as its template parameter. The constructor takes the callable object. It is a functor class, we use operator() to start executing the task, it also creates the promise object. get_future() returns the std::future object associated with the promise. std::packaged_task is move only class.
```
std::packaged_task<int(int, int)> ptask([](int a, int b){return a + b;});
//get the associated future
std::future<int> fut = ptask.get_future();

//Invoke the package in a seperate thread
std::thread thr(std::move(ptask), 6, 7);    //directly calling ptask(6,7); will invoke the task in this same thread

//Call get to wait and receive the result
std::cout << fut.get();

thr.join();
```
The advantages of using packaged_task is that we don't need to write the code to create the promise object and pass it to the task function, so the code will be cleaner and simpler.

## Async function
It provides a higher level abstraction than working with the std::thread. We can execute a task with std::async() which runs in the background. std::async returns a std::future object, which contains the result of the task.
```
unsigned long long fibonacci(unsigned long long n)
{
    if(n <= 1)
    {
        return 1;
    }
    
    //Code may throw a exception
    
    return fibonacci(n-1) + fibonacci(n-2);
}

int main()
{
    auto result = std::async(fibonacci, 44);
    
    //Do some work
    
    //Will throw a exception if the task throws one
    try
    {
        //Call get when we are ready, waits till the task is complete and the answer is ready
        int answer = result.get();
    }
    catch(std::exception& e)
    {
        //Handle the exception
    }
        
}
```
Like with std::promise and std::future, a task may throw a exception, it will be stored in the future object, it will be re-thrown when get is called.  
std::async() takes a additional parameter, launch flag. This is a flag that allows you to run the task in a new or the same thread. std::launch::async - a new thread is started for the task immediately, std::launch::deffered - the task is executed on the same thread when get() is called(lazy evaluation), we can use both options with | in that case the implementation will decide which option to use and this is the default.
```
int task()
{
    std::cout << "Task thread id : " << std::this_thread::get_id() << std::endl;
    std::this_thread::sleep_for(5s);
    return 42;
}

void func(const std::string& option = "default"s)
{
    std::future<int> result;
    if(option == "async"s)
    {
        std::async(std::launch::async, task);
    }
    else if(option == "deferred"s)
    {
        std::async(std::launch::deferred, task);
    }
    else
    {
        std::async(task);
    }
    
    std::this_thread::sleep_for(2s);
    std::cout << result.get() << std::endl; 
}

int main()
{
    std::cout << "Main thread id : " << std::this_thread::get_id() << std::endl;
    
    func("async");    //Task will run on a new thread immediately
    func("deferred"); //Task will run on the same thread when get() is called
    func("default");
}
```
std::launch::async is recommended to run a function asyncronously.  
wait() returns nothing, it just waits till the result is available on the std::future. wait_for() and wait_until() return std::future_status::ready if the result is available, return's std::future_status::timeout if timeout has expired, return's std::future_status::deferred if the result is being lazy evaluated. For lazy evaluation only get() makes sence as it starts the execution and waits for the result. 

## Choosing a thread object
We have 3 ways to execute a task std::thread object, std::packaged_task object and std::async.
* std::async() is the simplest way to execute a task, as it is a higher level abstraction on std::thread, the library manages the threads for the programmer, there is no need to use shared data. This also means that we don't have finer control of the inner std::thread object.
* std::packaged_task is the best choice when we want to represent tasks as objects. It is a lower level abstraction than std::async() hence we have more control on the task and on which thread to execute.
* std::thread is the most flexible. It allows to access the underlying thread implementation on the operating system. This allows to do things like setting thread priority, pin the thread to a particular core, these kinds of things are not possible on the other two.